<a href="https://colab.research.google.com/github/Onedory/yolov5-art-analyze/blob/main/%EB%9D%BC%EB%B2%A8%EB%B3%80%ED%99%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import json
import os

# 구글 드라이브 마운트
drive.mount('/content/drive')


Mounted at /content/drive


1. train데이터 라벨 변환


In [ ]:
from google.colab import drive
import json
import os

# 클래스 이름을 YOLO 클래스 ID로 매핑하는 딕셔너리
class_mapping = {
    "집전체": 0,
    "지붕": 1,
    "집벽": 2,
    "문": 3,
    "창문": 4,
    "굴뚝": 5,
    "연기": 6,
    "울타리": 7,
    "길": 8,
    "연못": 9,
    "산": 10,
    "나무": 11,
    "꽃": 12,
    "잔디": 13,
    "태양": 14,
    "나무전체": 15,
    "기둥": 16,
    "수관": 17,
    "가지": 18,
    "뿌리": 19,
    "나뭇잎": 20,
    "꽃": 21,
    "열매": 22,
    "그네": 23,
    "새": 24,
    "다람쥐": 25,
    "구름": 26,
    "달": 27,
    "별": 28,
    "사람전체": 29,
    "머리": 30,
    "얼굴": 31,
    "눈": 32,
    "코": 33,
    "입": 34,
    "귀": 35,
    "머리카락": 36,
    "목": 37,
    "상체": 38,
    "팔": 39,
    "손": 40,
    "다리": 41,
    "발": 42,
    "단추": 43,
    "주머니": 44,
    "운동화": 45,
    "여자구두": 46,
    "남자구두": 47
}

# 파일 경로 설정
json_folder_path = '/content/drive/MyDrive/Data/json'
label_folder_path = '/content/drive/MyDrive/Data/train/labels'

# 경로가 제대로 마운트되었는지 확인
if not os.path.exists(json_folder_path):
    raise FileNotFoundError(f"경로를 찾을 수 없습니다: {json_folder_path}")

if not os.path.exists(label_folder_path):
    os.makedirs(label_folder_path, exist_ok=True)

# JSON 파일 읽기 및 YOLO 포맷으로 변환
for json_file in os.listdir(json_folder_path):
    if json_file.endswith('.json'):
        with open(os.path.join(json_folder_path, json_file), 'r', encoding='utf-8') as f:
            data = json.load(f)

        image_width, image_height = 1280, 1280  # 이미지의 해상도

        annotations = data["annotations"]["bbox"]
        yolo_annotations = []

        for annotation in annotations:
            class_name = annotation["label"]
            if class_name not in class_mapping:
                print(f"라벨 '{class_name}'이(가) class_mapping에 정의되어 있지 않습니다.")
                continue
            class_id = class_mapping[class_name]

            x_center = (annotation["x"] + annotation["w"] / 2) / image_width
            y_center = (annotation["y"] + annotation["h"] / 2) / image_height
            width = annotation["w"] / image_width
            height = annotation["h"] / image_height

            yolo_annotations.append(f"{class_id} {x_center} {y_center} {width} {height}")

        # YOLO 포맷의 텍스트 파일로 저장
        label_file_name = os.path.splitext(json_file)[0] + '.txt'
        with open(os.path.join(label_folder_path, label_file_name), 'w', encoding='utf-8') as f:
            f.write('\n'.join(yolo_annotations))

print("YOLO 포맷으로 변환 완료")


YOLO 포맷으로 변환 완료


2. validation 데이터의 라벨변환


In [ ]:
from google.colab import drive
import json
import os
from tqdm import tqdm

# 클래스 이름을 YOLO 클래스 ID로 매핑하는 딕셔너리
class_mapping = {
    "집전체": 0,
    "지붕": 1,
    "집벽": 2,
    "문": 3,
    "창문": 4,
    "굴뚝": 5,
    "연기": 6,
    "울타리": 7,
    "길": 8,
    "연못": 9,
    "산": 10,
    "나무": 11,
    "꽃": 12,
    "잔디": 13,
    "태양": 14,
    "나무전체": 15,
    "기둥": 16,
    "수관": 17,
    "가지": 18,
    "뿌리": 19,
    "나뭇잎": 20,
    "꽃": 21,
    "열매": 22,
    "그네": 23,
    "새": 24,
    "다람쥐": 25,
    "구름": 26,
    "달": 27,
    "별": 28,
    "사람전체": 29,
    "머리": 30,
    "얼굴": 31,
    "눈": 32,
    "코": 33,
    "입": 34,
    "귀": 35,
    "머리카락": 36,
    "목": 37,
    "상체": 38,
    "팔": 39,
    "손": 40,
    "다리": 41,
    "발": 42,
    "단추": 43,
    "주머니": 44,
    "운동화": 45,
    "여자구두": 46,
    "남자구두": 47
}

# 파일 경로 설정
json_folder_path = '/content/drive/MyDrive/Data/validation/json'
label_folder_path = '/content/drive/MyDrive/Data/validation/labels'

# 경로가 제대로 마운트되었는지 확인
if not os.path.exists(json_folder_path):
    raise FileNotFoundError(f"경로를 찾을 수 없습니다: {json_folder_path}")

if not os.path.exists(label_folder_path):
    os.makedirs(label_folder_path, exist_ok=True)

# JSON 파일 읽기 및 YOLO 포맷으로 변환
json_files = [f for f in os.listdir(json_folder_path) if f.endswith('.json')]

for json_file in tqdm(json_files, desc="YOLO 포맷으로 변환 중"):
    with open(os.path.join(json_folder_path, json_file), 'r', encoding='utf-8') as f:
        data = json.load(f)

    image_width, image_height = 1280, 1280  # 이미지의 해상도

    annotations = data["annotations"]["bbox"]
    yolo_annotations = []

    for annotation in annotations:
        class_name = annotation["label"]
        if class_name not in class_mapping:
            print(f"라벨 '{class_name}'이(가) class_mapping에 정의되어 있지 않습니다.")
            continue
        class_id = class_mapping[class_name]

        x_center = (annotation["x"] + annotation["w"] / 2) / image_width
        y_center = (annotation["y"] + annotation["h"] / 2) / image_height
        width = annotation["w"] / image_width
        height = annotation["h"] / image_height

        yolo_annotations.append(f"{class_id} {x_center} {y_center} {width} {height}")

    # YOLO 포맷의 텍스트 파일로 저장
    label_file_name = os.path.splitext(json_file)[0] + '.txt'
    with open(os.path.join(label_folder_path, label_file_name), 'w', encoding='utf-8') as f:
        f.write('\n'.join(yolo_annotations))

print("YOLO 포맷으로 변환 완료")


YOLO 포맷으로 변환 중: 100%|██████████| 5600/5600 [02:59<00:00, 31.24it/s]

YOLO 포맷으로 변환 완료
